In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
import ipywidgets as widgets
import cv2
from tqdm.notebook import tqdm
from czifile import CziFile

%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../pytorch-unet-segmentation-master/src/")
sys.path.append("../pytorch-unet-segmentation-master/scripts/")
import LoadCZI
import Classic
import Tracking
path0 = "../pytorch-unet-segmentation-master/test2/test2.czi"
image_arrays = LoadCZI.load(path0)

TODO
- Calibrate h, templateWidowSize and serachWindowSize  (Denoising)
- Calibrate threshold (Thresholding)
- Understand how distances are calculated and what is the watershed marking

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(7, 5), tight_layout=True, sharex=True, sharey=True)

# Importar imágenes
r = c = 190
w = h = 388

def update(k, z, bandwidth):
    img = Classic.normalize_image(image_arrays[0, 0, 0, k, z, r:r+h, c:c+w, 0])
    ax[0, 0].imshow(img, cmap=plt.cm.Greys)
    # Denoising
    img = Classic.denoise_image(img, bandwidth=bandwidth)
    ax[0, 1].imshow(img, cmap=plt.cm.Greys)
    # Binarization
    binary_mask = Classic.get_binary_mask(img)
    ax[0, 2].imshow(binary_mask, cmap=plt.cm.Greys)
    binary_mask = ~binary_mask
    # Distance transform
    markers, watershed, dist_transform = Classic.do_watershed(img, binary_mask)
    ax[1, 0].imshow(dist_transform, cmap=plt.cm.Greys)
    # Watershed transform
    ax[1, 1].imshow(watershed, cmap=plt.cm.tab20)
    # Original with watershed borders on top
    img2 = img.copy()
    img2[watershed==-1] = 255
    ax[1, 2].imshow(img2, cmap=plt.cm.Greys)    
    

widgets.interact(update, 
                 k=widgets.IntSlider(min=0, max=59),
                 z=widgets.IntSlider(min=0, max=16),
                 bandwidth=widgets.FloatSlider(min=0., max=100., value=30, step=0.01)
                 );

### Watershed de dos frames consecutivos



In [ ]:
def watershed_pipeline(img, th=0.1):
    img = Classic.denoise_image(img)
    binary_mask = ~Classic.get_binary_mask(img)
    markers, watershed, dist_transform = Classic.do_watershed(img, binary_mask)
    return markers, watershed

z = 0
img1 = Classic.normalize_image(image_arrays[0, 0, 0, 0, z, r:r+h, c:c+w, 0])
marker1, water1 = watershed_pipeline(img1)
img2 = Classic.normalize_image(image_arrays[0, 0, 0, 1, z, r:r+h, c:c+w, 0])
marker2, water2 = watershed_pipeline(img2)

fig, ax = plt.subplots(2, 2, figsize=(7, 6), tight_layout=True)
ax[0, 0].imshow(img1, cmap=plt.cm.Greys)
ax[0, 1].imshow(img2, cmap=plt.cm.Greys)
ax[1, 0].imshow(water1, cmap=plt.cm.tab20)
ax[1, 1].imshow(water2, cmap=plt.cm.tab20)

# Hacer watershed a todas las imagenes

In [ ]:
img_list = []
water_list = []

for k in tqdm(range(0, 59)):
    img_list.append(Classic.normalize_image(image_arrays[0, 0, 0, k, z, r:r+h, c:c+w, 0]))
    marker, water = watershed_pipeline(img_list[-1], th=0.3)
    water_list.append(water)

# Medir la evolución del área de una de celula en particular

In [ ]:
import scipy.fft as sfft

cell_idx = 22
cell_idx_history, size_cell = Tracking.vecinosMasCercanos(water_list[0],water_list,cell_idx)
   
fig, ax = plt.subplots(2, 1, figsize=(6, 4), tight_layout=True)
ax[0].plot(size_cell)

freq = sfft.rfftfreq(n=len(size_cell))
S = sfft.rfft(size_cell - np.mean(size_cell))
ax[1].plot(freq, np.abs(S))

In [ ]:
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)

def update(k):
    ax.cla()
    img = img_list[k].copy()
    img[water_list[k]==cell_idx_history[k]] = 255
    ax.imshow(img, cmap=plt.cm.Greys)
    ax.set_title(k)
    
ani = FuncAnimation(fig, update, frames=range(59), repeat=False, interval=200)